### tf.data.TextLineDatasetのサンプル
https://deepage.net/tensorflow/2017/07/18/tensorflow-dataset-api.html

In [1]:
import tensorflow as tf
sess = tf.Session()

In [2]:
class Categories(object):
    __instance = None
    def __new__(cls, *args, **keys):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls)
        return cls.__instance

    def __init__(self):
            self.items = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9' ]
            self.num   = len(self.items)

In [3]:
import numpy as np
work_dir = '../colab/tmp'
input_file = work_dir + '/input.txt'
N = 100
num_labels = Categories().num

with open(input_file, 'w') as fp:
    for i in range(N):
        data = np.random.random(32).astype(np.float32)
        label = int(np.random.random()* num_labels) # ラベル番号:0-(num_labels-1)
        fp.writelines('{0}/{1:03d}.npy,{2}\n'.format(work_dir, i, label))
        np.save('{0}/{1:03d}.npy'.format(work_dir, i), data)

In [4]:
def to_index(label):
    return Categories().items.index(label)

def parse_csv(line):
    [filename, category] = line.decode('utf-8').split(',')
    return filename, to_index(category)

def read_data(filename, label):
    inputs = np.load(filename.decode('utf-8'))#.astype(np.float32)
    return inputs, label

def one_hot(data, label):
    return data, tf.one_hot( label, Categories().num  )

dataset = tf.data.TextLineDataset(input_file)
#dataset = dataset.skip(1)   # 列の読み飛ばし
dataset = dataset.map(lambda x  : tf.py_func(parse_csv, [x],    [tf.string,  tf.int32]))
dataset = dataset.map(lambda x,y: tf.py_func(read_data, [x, y], [tf.float32, tf.int32]))
dataset = dataset.map(one_hot)
dataset = dataset.repeat()
dataset = dataset.shuffle(4)
dataset = dataset.batch(4)

iterator = dataset.make_one_shot_iterator()
next_elem = iterator.get_next()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [6]:
val = sess.run(next_elem)

print(len(val))
print('\ndata ----------------------------------')
print(val[0].shape)
#print(val[0])

print('\nlabels ----------------------------------')
print(val[1].shape)
print(val[1])

2

data ----------------------------------
(4, 32)

labels ----------------------------------
(4, 10)
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
